In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mne
import os

Checking out the information about the data

In [5]:
data_path = "Preprocessed_data/sub_003" #path to the preprocessed data

list_files = os.listdir(data_path) #listing the files in the data folder

print("The files", list_files)

food_data = np.load(os.path.join(data_path, 'food_dec.npy'))

print("the shape is:", food_data.shape)



The files ['positive_1_dec.npy', 'positive_2_dec.npy', 'food_dec.npy', 'food_2_dec.npy', 'neutral_2_dec.npy', 'neutral_1_dec.npy', 'food_1_dec.npy', 'nonfood_dec.npy']
the shape is: (194, 246, 111)


Pseudo-Trial Creation

In [6]:
#number of groups =  5
#number of permutations = 20

def create_pseudo_trials(data, n_groups = 5, n_permutations = 20):
    """
    Create pseudo-trials by averaging trials within each random group

    Parameters:
    -data: food_dec numpy array
    -n_groups: number of pseudo-trials group to create
    -n_permutations: number of random permutations

    Output:
    -pseudotrials - The generated pseudo-trials with shape (n_permutations, n_groups, n_channels, n_times)
    
    
    """
    n_trials, n_channels, n_times = data.shape
    pseudo_trials = []

    for random_perm in range(n_permutations):

        
        permutated_indices = np.random.permutation(n_trials) #this first generates random positions for the n_trials

       
        groups = [permutated_indices[i::n_groups] for i in range(n_groups)] #this means it takes every 5th element till it gets to the end of the shuffled permutated indices, but the initial starting point is determined by i 

        #average the trials in each group
        averaged_group = np.array([data[group].mean(axis = 0) for group in groups])

        #update the pseudotrials list
        pseudo_trials.append(averaged_group)
    
    return np.array(pseudo_trials)



Next is to loop through each subject, create pseudo traisl for food and non-food conditions

In [10]:
def process_all_subjects(data_dir, output_dir, n_groups = 5, n_permutations = 20):

    """
    Create pseudotrials for each subject and each condition of interest

    Parameters:
    -data_dir: path to the directory that contains the data
    -output_dir: path to the directory that contains the created pseudo trials for each subject
    -n_groups: the number of pseudo-trial groups
    -n_permutations: the number of random permutations
    
    """

    os.makedirs(output_dir, exist_ok=True) #if the folder does not exist, it will create a folder

    #looping through each subject's folder
    for subject in os.listdir(data_dir):
        subject_path = os.path.join(data_dir, subject)

        if os.path.isdir(subject_path) and subject.startswith("sub_"): #this is to check if the subject path is a directory
            print(f"Processing for subject: {subject}")


            # Create subject's output directory
            subject_output_dir = os.path.join(output_dir, subject)
            os.makedirs(subject_output_dir, exist_ok=True)

            # Define paths to food_dec.npy and nonfood_dec.npy
            food_data_path = os.path.join(subject_path, "food_dec.npy")
            nonfood_data_path = os.path.join(subject_path, "nonfood_dec.npy")

            #this part processes the food_dec.npy for each subject

            food_output_path = os.path.join(subject_output_dir, "food_pseudo_trials.npy")
            if os.path.exists(food_data_path) and not os.path.exists(food_output_path):
                    
                    print(f"Creating pseudotrials for Food Condition for Subject {subject}...")
                    food_data = np.load(food_data_path) #this loads the data from the path
                    food_pseudo_trials = create_pseudo_trials(food_data, n_groups, n_permutations) #this calls the function to create pseudo trials
                    np.save(food_output_path, food_pseudo_trials) #this saves the pseudo-trials to the folder
                    print(f"Saved food pseudo trials for subject {subject}")
            elif os.path.exists(food_output_path):
                    print(f"Food pseudo-trials already exist for subject {subject}. Skipping...")

            
            #this part processes the nonfood_dec.npy for each subject
            nonfood_output_path = os.path.join(subject_output_dir, "nonfood_pseudo_trials.npy")
            if os.path.exists(nonfood_data_path) and not os.path.exists(nonfood_output_path):

                    print(f"Creating pseudo-trials for Non-Food condition for subject {subject}...")
                    nonfood_data = np.load(nonfood_data_path)
                    nonfood_pseudo_trials = create_pseudo_trials(nonfood_data, n_groups, n_permutations) #this calls the function to create pseudo trials
                    np.save(nonfood_output_path,nonfood_pseudo_trials) #this saves the pseudo-trials to the folder
                    print(f"Saved Nonfood pseudo trials for subject {subject}")
            elif os.path.exists(nonfood_output_path):
                    print(f"Non-Food pseudo-trials already exist for subject {subject}. Skipping...")

            print(f"Finished processing subject {subject}.\n")
        else:
            print(f"Skipping non-subject folder: {subject}")



In [11]:
data_dir = "/Users/folasewaabdulsalam/MEG_Decoding/Preprocessed_data"
output_dir = "/Users/folasewaabdulsalam/MEG_Decoding/pseudo_trials"

process_all_subjects(data_dir, output_dir)

Processing for subject: sub_045
Creating pseudotrials for Food Condition for Subject sub_045...
Saved food pseudo trials for subject sub_045
Creating pseudo-trials for Non-Food condition for subject sub_045...
Saved Nonfood pseudo trials for subject sub_045
Finished processing subject sub_045.

Processing for subject: sub_011
Creating pseudotrials for Food Condition for Subject sub_011...
Saved food pseudo trials for subject sub_011
Creating pseudo-trials for Non-Food condition for subject sub_011...
Saved Nonfood pseudo trials for subject sub_011
Finished processing subject sub_011.

Processing for subject: sub_016
Creating pseudotrials for Food Condition for Subject sub_016...
Saved food pseudo trials for subject sub_016
Creating pseudo-trials for Non-Food condition for subject sub_016...
Saved Nonfood pseudo trials for subject sub_016
Finished processing subject sub_016.

Processing for subject: sub_029
Creating pseudotrials for Food Condition for Subject sub_029...
Saved food pseud

In [12]:
#let us inspect the food pseudo_trials
data = np.load("/Users/folasewaabdulsalam/MEG_Decoding/pseudo_trials/sub_003/food_pseudo_trials.npy")
data.shape

(20, 5, 246, 111)

Next, we will move to dimensionality reduction using PCA

--Load the pseudo-trials
---reshape for pca
---apply pca
----reshape back
----save

In [13]:
from sklearn.decomposition import PCA

In [64]:
def pca_all_time_point(input_dir, output_dir, n_components=20):
    """
    Compute PCA using the all-time point approach.

    Parameters:
    - input_dir: directory containing pseudo-trials
    - output_dir: directory to save PCA-transformed data
    - n_components: number of PCA components to retain
    """
    os.makedirs(output_dir, exist_ok=True)

    for subject in os.listdir(input_dir):
        subject_path = os.path.join(input_dir, subject)
        if not os.path.isdir(subject_path):
            continue

        if not subject.startswith("sub"):
            print(f"Skipping non-subject directory: {subject}")
            continue

        print(f"Processing PCA for subject: {subject}...")

        subject_output_dir = os.path.join(output_dir, subject)
        os.makedirs(subject_output_dir, exist_ok=True)

        conditions_list = ["food_pseudo_trials.npy", "nonfood_pseudo_trials.npy"]
        for condition in conditions_list:
            condition_path = os.path.join(subject_path, condition)
            output_path = os.path.join(subject_output_dir, f"{condition.split('.')[0]}_pca.npy")

            if os.path.exists(output_path):
                print(f"PCA-transformed data already exists for {condition} in {subject}. Skipping...")
                continue

            if os.path.exists(condition_path):
                print(f"Applying PCA for condition: {condition} in {subject}...")

                pseudo_trials = np.load(condition_path)
                n_permutations, n_groups, n_channels, n_times = pseudo_trials.shape

                # Flatten for PCA
                reshaped_data = pseudo_trials.transpose(0, 1, 3, 2).reshape(-1, n_channels)
                print(f"Shape after flattening for PCA: {reshaped_data.shape}")

                # Apply PCA
                pca = PCA(n_components=n_components)
                pca_transformed = pca.fit_transform(reshaped_data)
                print(f"Shape after PCA: {pca_transformed.shape}")

                # Reshape PCA output to include n_groups
                pca_transformed_grouped = pca_transformed.reshape(n_permutations, n_groups, n_times, n_components)
                print(f"Shape with groups retained: {pca_transformed_grouped.shape}")

                # Aggregate over groups
                pca_transformed_final = pca_transformed_grouped.mean(axis=1)
                print(f"Shape after aggregation: {pca_transformed_final.shape}")

                # Transpose for final output
                pca_transformed_final = pca_transformed_final.transpose(0, 2, 1)
                print(f"Final PCA-transformed shape: {pca_transformed_final.shape}")

                # Save the PCA-transformed data
                np.save(output_path, pca_transformed_final)
                print(f"Saved PCA-transformed data to: {output_path}")

            else:
                print(f"Condition file {condition} not found for subject {subject}. Skipping...")


In [65]:
input_dir = "/Users/folasewaabdulsalam/MEG_Decoding/pseudo_trials"
output_dir = "/Users/folasewaabdulsalam/MEG_Decoding/pca_transformed_data"

pca_all_time_point(input_dir, output_dir)

Processing PCA for subject: sub_045...
Applying PCA for condition: food_pseudo_trials.npy in sub_045...
Shape after flattening for PCA: (11100, 246)
Shape after PCA: (11100, 20)
Shape with groups retained: (20, 5, 111, 20)
Shape after aggregation: (20, 111, 20)
Final PCA-transformed shape: (20, 20, 111)
Saved PCA-transformed data to: /Users/folasewaabdulsalam/MEG_Decoding/pca_transformed_data/sub_045/food_pseudo_trials_pca.npy
Applying PCA for condition: nonfood_pseudo_trials.npy in sub_045...
Shape after flattening for PCA: (11100, 246)
Shape after PCA: (11100, 20)
Shape with groups retained: (20, 5, 111, 20)
Shape after aggregation: (20, 111, 20)
Final PCA-transformed shape: (20, 20, 111)
Saved PCA-transformed data to: /Users/folasewaabdulsalam/MEG_Decoding/pca_transformed_data/sub_045/nonfood_pseudo_trials_pca.npy
Processing PCA for subject: sub_011...
Applying PCA for condition: food_pseudo_trials.npy in sub_011...
Shape after flattening for PCA: (11100, 246)
Shape after PCA: (1110

In [63]:
#let's check the shape of one of the pca_transformed data

data = np.load("/Users/folasewaabdulsalam/MEG_Decoding/pca_transformed_data/sub_003/food_pseudo_trials_pca.npy")
data.shape

(20, 5, 20, 111)